In [29]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# import libraies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from dataprep.eda import create_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score,f1_score,recall_score,precision_score,classification_report
from lightgbm import LGBMClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# Reading Data

In [2]:
# importing the data.
raw_data = pd.read_csv('../data/train.csv')
raw_data.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [3]:
raw_data.info()
""" 
17996 observation
17 variable
Artist Name, and Track Name are of type str, while the rest are numerical variables.
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         17996 non-null  object 
 1   Track Name          17996 non-null  object 
 2   Popularity          17568 non-null  float64
 3   danceability        17996 non-null  float64
 4   energy              17996 non-null  float64
 5   key                 15982 non-null  float64
 6   loudness            17996 non-null  float64
 7   mode                17996 non-null  int64  
 8   speechiness         17996 non-null  float64
 9   acousticness        17996 non-null  float64
 10  instrumentalness    13619 non-null  float64
 11  liveness            17996 non-null  float64
 12  valence             17996 non-null  float64
 13  tempo               17996 non-null  float64
 14  duration_in min/ms  17996 non-null  float64
 15  time_signature      17996 non-null  int64  
 16  Clas

' \n17996 observation\n17 variable\nArtist Name, and Track Name are of type str, while the rest are numerical variables.\n'

In [4]:
#auto_report = create_report(raw_data, title="Auto EDA report")

In [5]:
#auto_report.save(path="../report/Auto_EDA_report.html")

# Data Imputing

In [6]:
raw_data.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [7]:
# filling missing value with zeros. 
#df = raw_data.fillna(0)

In [8]:
# impute using the mean of each column (other option)
df = raw_data
df['Popularity'].fillna(raw_data['Popularity'].mean(), inplace=True) # imputing the popularity column with the mean
df['key'].fillna(raw_data['key'].mean(), inplace=True) # imputing the key with the mean 
df['instrumentalness'].fillna(raw_data['instrumentalness'].mean(), inplace=True) # imputing the instrumentalness with mean

df.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Class                 0
dtype: int64

In [9]:
df.head(2)

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.0171,0.177562,0.0849,0.899,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.0011,0.004010,0.1010,0.569,116.454,251733.0,4,10


# Additional Attributes

In [10]:
df['Artist_Length'] = df['Artist Name'].apply(len) 
df['Artist_Char_Count'] = df['Artist Name'].str.split().str.len()     
df['Track_Length'] = df['Track Name'].apply(len)   
df['Track_Char_Count'] = df['Track Name'].str.split().str.len() 
df['Track_Digit'] = df['Track Name'].str.findall(r'[0-9]').str.len() 

In [11]:
df['Track_in_min']= df['duration_in min/ms']/60000
df['danceability_by_artist']= df.groupby(['Artist Name'])['danceability'].transform('mean')
df['energy_by_artist']= df.groupby(['Artist Name'])['energy'].transform('mean')
df['loudness_by_artist']= df.groupby(['Artist Name'])['loudness'].transform('mean')
df['acousticness_by_artist']= df.groupby(['Artist Name'])['acousticness'].transform('mean')

# Label encoding 

In [12]:
# Encoding categorical variables.
encoder = LabelEncoder()

# Enconding artist name
df['Artist Name'] = encoder.fit_transform(df['Artist Name'])

In [13]:
# Encoding track name
df['Track Name'] = encoder.fit_transform(df['Track Name'])

# or remove it all together
df.drop('Track Name', axis=1, inplace=True)

## Data Splitting (Without over sampling the data)

In [14]:
column_names= ['Artist Name', 'Track Name', 'Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature',  'Artist_Length',
       'Artist_Char_Count', 'Track_Length', 'Track_Char_Count', 'Track_Digit',
       'Track_in_min', 'danceability_by_artist', 'energy_by_artist',
       'loudness_by_artist', 'acousticness_by_artist']

In [15]:
# Standard Scaler
def scale_features(X_train, X_test):
    scaler = StandardScaler().fit(X_train)
    Xtrain = scaler.transform(X_train)
    Xtest = scaler.transform(X_test)
    return Xtrain, Xtest

# Performance metrics
def Performance_Metrics(y_test,y_pred):
    print('-------------------PERFORMANCE METRICS-------------------')
    print("Accuracy:",100*accuracy_score(y_test, y_pred))
    print('F1 score:', 100*f1_score(y_test, y_pred,average='weighted'))
    print('Recall:', 100*recall_score(y_test, y_pred,average='weighted'))
    print('Precision:', 100*precision_score(y_test, y_pred, average='weighted'))
    print('---------------------------------------------------------')

# Function used for displaying confusion matrix.
def display_confusionMatrix(clf, X, y, title):
  cm = confusion_matrix(y, clf.predict(X), labels=clf.classes_)
  normed_c = (cm.T / cm.astype(np.float).sum(axis=1)).T
  disp = ConfusionMatrixDisplay(confusion_matrix=normed_c,
                                display_labels=clf.classes_)
  disp.plot(
      cmap=plt.cm.Blues
      )
  disp.ax_.set_title(title + " Confusion Matrix:")

In [16]:
X = df.drop('Class', axis=1)
y = df[['Class']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)
X_train_norm,X_test_norm=scale_features(X_train, X_test)

# Model training (Without over sampling)

In [17]:
# Classifier K-Nearest Neighbors (KN)
classifier_kn = KNeighborsClassifier()
title_kn = "K-Nearest Neighbor"

# LGBM
classifier_lg = LGBMClassifier()
title_lg = "LGBM"

# Random Forest
classifier_rf = RandomForestClassifier()
title_rf = "Random Forest"

# Logistic Regression
classifier_lr = LogisticRegression(max_iter=500,multi_class='multinomial')
title_lr = "Logistic Regression"

# Multi-Layer-Perceptron 
classifier_mlp = MLPClassifier()
title_mlp = "Multi-Layer-Perceptron"

# SVM
classifier_svc = SVC()
title_svc = "Support Vector Machine"

# XGBoost
classifier_xgb=xgb.XGBClassifier(random_state=42,use_label_encoder=False)
title_xgb = "XGBoost"

classifiers = [classifier_lg,classifier_lr, classifier_kn, classifier_mlp, classifier_rf, classifier_svc]

titles = [title_lg,title_lr,title_kn, title_mlp,title_rf, title_svc]

In [18]:
# Fitting training data on-to the classifiers
for clf, title in zip(classifiers, titles):
  clf.fit(X_train_norm, y_train)
  prediction = clf.predict(X_test_norm)
  #Performance_Metrics(y_test,prediction)
  print(title + " Accuracy Score:", round(clf.score(X_test_norm, y_test), 2))

LGBM Accuracy Score: 0.59
Logistic Regression Accuracy Score: 0.53
K-Nearest Neighbor Accuracy Score: 0.46
Multi-Layer-Perceptron Accuracy Score: 0.55
Random Forest Accuracy Score: 0.56
Support Vector Machine Accuracy Score: 0.57


# Model training (with over sampling)

In [24]:
X = df.drop('Class', axis=1)
y = df[['Class']]

X_train_norm,X_test_norm=scale_features(X_train, X_test)

oversample = SMOTE()
X_over, y_over = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_over , y_over , test_size=0.2, random_state=42, shuffle=True )

In [25]:
# Fitting training data on-to the classifiers
for clf, title in zip(classifiers, titles):
  clf.fit(X_train_norm, y_train)
  prediction = clf.predict(X_test_norm)
  #Performance_Metrics(y_test,prediction)
  print(title + " Accuracy Score:", round(clf.score(X_test_norm, y_test), 2))

LGBM Accuracy Score: 0.8
Logistic Regression Accuracy Score: 0.65
K-Nearest Neighbor Accuracy Score: 0.75
Multi-Layer-Perceptron Accuracy Score: 0.74
Random Forest Accuracy Score: 0.8
Support Vector Machine Accuracy Score: 0.74


# Hyperparameter Tuning 

In [26]:
# Parameter tuning function, and printing the best score, parameter, and estimator
def parameter_tuning(model, hyper_parameter_grid, X_train, y_train ,X_test,y_test):
  # Initialising grid search
  grid = RandomizedSearchCV(estimator=model,
                      param_distributions =  hyper_parameter_grid,
                      scoring='accuracy',
                      verbose=2,
                      cv = 2,
                      n_jobs=-1
                      )
  
  # Inputing the data onto the grid search
  result = grid.fit(X_train, y_train)

  # Displaying the best score with its corrsponding parameters
  print('Best Score: ', result.best_score_)
  print('Best Params: ', result.best_params_)
  print('Best Estimator: ',result.best_estimator_)
  prediction = result.predict(X_test)
  Performance_Metrics(y_test,prediction)

### LGBM

In [30]:
lg_param_test ={'num_leaves': sp_randint(2, 40), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

lg = LGBMClassifier()
parameter_tuning(lg, lg_param_test, X_train_norm, y_train ,X_test_norm,y_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

[CV] END colsample_bytree=0.7471481862906044, min_child_samples=480, min_child_weight=1000.0, num_leaves=2, reg_alpha=5, reg_lambda=10, subsample=0.6624774399591082; total time=   1.7s
[CV] END colsample_bytree=0.8429312936767794, min_child_samples=221, min_child_weight=10000.0, num_leaves=32, reg_alpha=0.1, reg_lambda=0, subsample=0.9897686028385879; total time=   1.8s
[CV] END colsample_bytree=0.7471481862906044, min_child_samples=480, min_child_weight=1000.0, num_leaves=2, reg_alpha=5, reg_lambda=10, subsample=0.6624774399591082; total time=   1.8s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

[CV] END colsample_bytree=0.8429312936767794, min_child_samples=221, min_child_weight=10000.0, num_leaves=32, reg_alpha=0.1, reg_lambda=0, subsample=0.9897686028385879; total time=   2.2s


...

[CV] END colsample_bytree=0.6568619147826495, min_child_samples=169, min_child_weight=0.001, num_leaves=8, reg_alpha=0, reg_lambda=20, subsample=0.403137036266489; total time=   5.0s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END colsample_bytree=0.6568619147826495, min_child_samples=169, min_child_weight=0.001, num_leaves=8, reg_alpha=0, reg_lambda=20, subsample=0.403137036266489; total time=   4.9s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END colsample_bytree=0.58786320985637, min_child_samples=188, min_child_weight=10.0, num_leaves=22, reg_alpha=100, reg_lambda=20, subsample=0.7581705813915645; total time=   3.4s
[CV] END colsample_bytree=0.58786320985637, min_child_samples=188, min_child_weight=10.0, num_leaves=22, reg_alpha=100, reg_lambda=20, subsample=0.7581705813915645; total time=   3.8s
[CV] END colsample_bytree=0.6172787853939783, min_child_samples=281, min_child_weight=1, num_leaves=13, reg_alpha=1, reg_lambda=5, subsample=0.540680878571639; total time=   5.2s
[CV] END colsample_bytree=0.6172787853939783, min_child_samples=281, min_child_weight=1, num_leaves=13, reg_alpha=1, reg_lambda=5, subsample=0.540680878571639; total time=   4.9s
[CV] END colsample_bytree=0.5163302358366071, min_child_samples=265, min_child_weight=0.01, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1, subsample=0.35611515446661157; total time=   6.2s
[CV] END colsample_bytree=0.5163302358366071, min_child_samples=265, min_child_weight=

### SVM

In [32]:
svm_param_grid = {'C': [0.1,1, 10, 100], 
              'gamma': [1,0.1,0.01,0.001],
              'kernel': ['rbf', 'poly', 'sigmoid']}

svm_model = SVC()
parameter_tuning(svm_model , svm_param_grid, X_train_norm, y_train ,X_test_norm,y_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

[CV] END ...................C=100, gamma=0.1, kernel=sigmoid; total time= 1.1min
[CV] END ...................C=100, gamma=0.1, kernel=sigmoid; total time= 1.1min


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END .......................C=1, gamma=0.01, kernel=poly; total time= 1.2min


...

[CV] END .........................C=100, gamma=1, kernel=rbf; total time= 2.4min


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END .........................C=100, gamma=1, kernel=rbf; total time= 2.4min


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ..................C=100, gamma=0.01, kernel=sigmoid; total time=  37.1s
[CV] END ..................C=100, gamma=0.01, kernel=sigmoid; total time=  39.1s
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time=  59.9s
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time=  56.7s
[CV] END ......................C=1, gamma=0.001, kernel=poly; total time= 1.2min
[CV] END ......................C=1, gamma=0.001, kernel=poly; total time= 1.2min
[CV] END .........................C=10, gamma=1, kernel=poly; total time= 3.9min
[CV] END .........................C=10, gamma=1, kernel=poly; total time= 4.1min
Best Score:  0.7621180123516257
Best Params:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 100}
Best Estimator:  SVC(C=100, gamma=0.1)
-------------------PERFORMANCE METRICS-------------------
Accuracy: 80.18001469507715
F1 score: 79.8720837760752
Recall: 80.18001469507715
Precision: 79.79445417200164
---------------------------------------------------------

-------------------PERFORMANCE METRICS-------------------

Accuracy: 80.18001469507715

F1 score: 79.8720837760752

Recall: 80.18001469507715

Precision: 79.79445417200164


### KNN

In [33]:
parameters = { 'n_neighbors' : [3,5,7,9,11,13,15],
               'metric' : ['minkowski','euclidean','manhattan']}
neigh = KNeighborsClassifier()
grid_neigh = GridSearchCV(neigh, parameters)
grid_neigh.fit(X_train_norm, y_train) 
Y_pred=grid_neigh.predict(X_test_norm)
accuracy=grid_neigh.score(X_test_norm,y_test)
Performance_Metrics(y_test,Y_pred)

-------------------PERFORMANCE METRICS-------------------
Accuracy: 79.09625275532697
F1 score: 77.77472215062097
Recall: 79.09625275532697
Precision: 79.46170751427233
---------------------------------------------------------


-------------------PERFORMANCE METRICS-------------------

Accuracy: 79.09625275532697

F1 score: 77.77472215062097

Recall: 79.09625275532697

Precision: 79.46170751427233


### Random Forest Classifer

In [35]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 4, stop = 100, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3,6,7,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3 , 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
RF_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [37]:
crf= RandomForestClassifier()
parameter_tuning(crf, RF_grid, X_train_norm, y_train ,X_test_norm,y_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `ma

[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=7, n_estimators=4; total time=   0.4s
[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=7, n_estimators=4; total time=   0.4s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/si

[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=7, n_estimators=28; total time=   2.8s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=7, n_estimators=28; total time=   2.9s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=7, n_estimators=28; total time=   3.0s


...

[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=100; total time=  10.4s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=76; total time=   7.6s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=76; total time=   8.3s


/home/ibrahim/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=3, n_estimators=28; total time=   2.1s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=3, n_estimators=28; total time=   2.1s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=6, n_estimators=76; total time=   6.3s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=6, n_estimators=76; total time=   6.4s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  14.3s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  14.7s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=7, n_estimators=100; total time=   8.0s
[CV] END bootstrap=True, max_depth=80, max_feature

-------------------PERFORMANCE METRICS-------------------

Accuracy: 81.05253490080823

F1 score: 80.48892035690088

Recall: 81.05253490080823

Precision: 80.20424477501095
